# ENDEPORTE_PROYECTO_FINAL

### Objetivo:
Mediante tecnicas de machine learning establecer las variables que mayor relacion tienen con la desercion e identificar los estudiantes de carreras profesionales o tecnologicas de la entidad que pueden desertar o no de sus programas.

### Contexto del negocio
Para la entidad es dificil identificar tempranamente los estudiantes que están en riesgo de abandonar sus carreras, las intervenciones son a menudo menos efectivas porque las señales de riesgo no aparecen hasta que el estudiante ya ha tomado la decisión de desertar. Se espera que mediante modelos de machine learning se identifiquen con anticipacion los estudiantes que pueden llegar a desertar con el fin de brindar a cada estudiante el tipo y nivel de apoyo que necesita para superar sus dificultades.

In [2]:
!pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\andre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
from sklearn.preprocessing import LabelEncoder # Import LabelEncoder



##**1. Entendimiento y preparación de los datos:**


### Cargue del dataset

In [4]:
demograficos_url="https://github.com/amrios80/Endeporte/raw/refs/heads/main/data/Demograficos.csv.gz"
historicos_url="https://github.com/amrios80/Endeporte/raw/refs/heads/main/data/HistoricoNotas.csv.gz"

demograficos_df = pd.read_csv(demograficos_url, sep=";", compression='gzip')
historicos_df=pd.read_csv(historicos_url, sep=";", compression="gzip")

C:\Users\andre\AppData\Local\Temp\ipykernel_16084\842882547.py:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  historicos_df=pd.read_csv(historicos_url, sep=";", compression="gzip")


### Limpieza de datos

* ***Estandarizacion*** : Segun las reglas acordadas con ENDEPORTE, Se excluyen del estudio las ESPECIALIZACIONES:

In [5]:
demograficos_df.sample(3)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION
230,2017 - 2G,7305172G010,CERES SALTO AFRO GUACHENE CAUCA,7305,TECNOLOGIA EN DEPORTE,1,4,Masculino,13-08-1996,1.0,GUACHENE,CAUCA,Soltero,4,EXCLUIDO CANCELACION SEMESTRE,REINGRESO,ACTIVO,NUEVO REGULAR,NINGUNA,NINGUNA,29/12/17,19300.0,GUACHENE,19.0,CAUCA,0.0,1.0,1,2017-II,21,Edad entre 18 y 25 años,Fac. Deporte,Tecnológico
15066,2019 - 1,7306191165,ESCUELA NACIONAL DEL DEPORTE,7306,FISIOTERAPIA,1,"4,1",Femenino,10-03-2001,3.0,CALI,VALLE DEL CAUCA,Soltero,"4,1",ACTIVO,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,16/01/19,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2019-I,18,Edad entre 18 y 25 años,Fac. Salud,Universitrario
29949,2021 - 1,53212171107,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,8,"4,5",Masculino,20-10-1994,3.0,CALI,VALLE DEL CAUCA,Soltero,4,EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,28/01/21,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2021-I,27,Edad entre 26 y 33 años,Fac. Deporte,Universitrario


In [6]:
demograficos_df["NIVEL_FORMACION"].value_counts()

NIVEL_FORMACION
Universitrario     48022
Tecnológico         4994
Especialización     2403
Name: count, dtype: int64

In [7]:
demograficos_df=demograficos_df[(demograficos_df["NIVEL_FORMACION"] != "Especialización")]



*  ***Eliminacion de codigos de estudiantes duplicados*** : La informacion demografica debe estar a nivel de estudiante, segun lo definido con la entidad, se valida que no haya duplicidad a nivel de codigo del estudiante.



In [8]:
total_rows = demograficos_df.shape[0]
total_rows

53016

In [9]:
demograficos_df.loc[demograficos_df.duplicated(subset=["CODIGO"], keep=False)].sample(3)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION
33301,2021 - 2,53212172308,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,4,"3,8",Masculino,10-12-1996,2.0,PASTO,NARIÑO,Soltero,"3,1",ACTIVO,ANTIGUO,ACTIVO,REINGRESO,NINGUNA,NINGUNA,23/07/21,52001.0,PASTO,52.0,NARIÑO,0.0,NaN,1,2021-II,25,Edad entre 18 y 25 años,Fac. Deporte,Universitrario
46870,2023 - 1,104024231098,ESCUELA NACIONAL DEL DEPORTE,104024,TERAPIA OCUPACIONAL,1,"3,8",Femenino,28-08-2004,2.0,CALI,VALLE DEL CAUCA,Soltero,"3,8",ACTIVO,ANTIGUO,ACTIVO,NUEVO REGULAR,NINGUNA,NINGUNA,20/01/23,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2023-I,19,Edad entre 18 y 25 años,Fac. Salud,Universitrario
54917,2020 - 1,7306182036,ESCUELA NACIONAL DEL DEPORTE,7306,FISIOTERAPIA,4,"4,1",Femenino,15-10-2001,3.0,CALI,VALLE DEL CAUCA,Soltero,"3,7",ACTIVO,ANTIGUO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,27/12/19,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2020-I,18,Edad entre 18 y 25 años,Fac. Salud,Universitrario


In [10]:
demograficos_df[demograficos_df["CODIGO"]=="102383181022"].sort_values(by="PERIODO_ACAENICO",ascending=False).head(3)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION
38163,2022 - 1,102383181022,ESCUELA NACIONAL DEL DEPORTE,102383,NUTRICION Y DIETETICA,7,"4,6",Femenino,06-04-2000,3.0,CALI,VALLE DEL CAUCA,Soltero,"4,2",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,16/02/22,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2022-I,22,Edad entre 18 y 25 años,Fac. Salud,Universitrario
34325,2021 - 2,102383181022,ESCUELA NACIONAL DEL DEPORTE,102383,NUTRICION Y DIETETICA,8,"4,2",Femenino,06-04-2000,3.0,CALI,VALLE DEL CAUCA,Soltero,"4,1",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,30/07/21,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2021-II,21,Edad entre 18 y 25 años,Fac. Salud,Universitrario
30500,2021 - 1,102383181022,ESCUELA NACIONAL DEL DEPORTE,102383,NUTRICION Y DIETETICA,7,"4,4",Femenino,06-04-2000,3.0,CALI,VALLE DEL CAUCA,Soltero,"3,9",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,12/01/21,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2021-I,21,Edad entre 18 y 25 años,Fac. Salud,Universitrario


Se observa que el codigo de estudiante se repite tantas veces como periodos académicos haya cursado y las variables de semestre, promediosemestre, promediogeneral y fecha_matricula son las correspondientes al periodo academico, las demas variables demograficas son las mismas para todos los periodos academicos. Por lo anterior, se tomara la informacion demografica correspondiente al ultimo periodo academico.

In [11]:
idx = demograficos_df.groupby('CODIGO')['PERIODO_ACAENICO'].idxmax()

In [12]:
max_periodo_rows = demograficos_df.loc[idx]

In [13]:
demograficos_df_uniq=max_periodo_rows.copy()

In [14]:
total_rows_uniq = demograficos_df_uniq.shape[0]
total_rows_uniq

10021

El estudio se realizara con la informacion demografica correspondiente al ultimo periodo academico de 10.021 estudiantes unicos.

*  ***Calculo de la variable objetivo (Desercion)*** : Si la variable denominada “situación actual” contiene cumple las siguientes condiciones “excluido cancelación semestre” o “excluido no renovación matricula” o “retiro definitivo” y la variable denominada “categoría actual” el valor es diferente de “continuidad académica - egresado” se considera “Deserción”.

In [15]:
def calculate_target(row):
    if (row['SITUACION_ACTUAL'] == 'EXCLUIDO NO RENOVACION DE MATRICULA' or row["SITUACION_ACTUAL"]=='RETIRO DEFINITIVO' or row["SITUACION_ACTUAL"]=='EXCLUIDO CANCELACION SEMESTRE' ) and  (row['CATEGORIA_ACTUAL'] != "CONTINUIDAD ACADEMICA - EGRESADO") :
        return 0
    else:
        return 1

In [16]:
demograficos_df_uniq["target"]=demograficos_df_uniq.apply(calculate_target, axis=1)

In [17]:
demograficos_df_uniq["target"].value_counts()

target
1    8760
0    1261
Name: count, dtype: int64

Se observa que 1.261 estudiantes de 10.021 desertan, es decir la tasa de desercion de la poblacion estudiada es de 12,6% valor que coincide con las cifras de desercion reportadas por la entidad.

* ***Creacion del dataset (features + target) :*** De acuerdo al analisis exploratorio desarrollado para la primera entrega, se tendran en cuenta para el estudio las siguientes variables, dada su relacion directa con la desercion (target): ESTRATO, SEMESTRE, PROMEDIOSEMESTRE, EDAD2, PROGRAMA, GENERO, CIUDADRESIDENCIA, CIUDADNACIMIENTO. Con base en estas variables de entrada se creara el dataset previa conversion numerica de las variables: ESTRATO, SEMESTRE, PROMEDIOSEMESTRE, EDAD2 y codificacion de las variables categoricas: PROGRAMA, GENERO, CIUDADRESIDENCIA, CIUDADNACIMIENTO.

Conversion a valor numerico del ESTRATO, SEMESTRE, PROMEDIOSEMESTRE Y EDAD2

In [18]:
field="ESTRATO"
demograficos_df_uniq[field] = pd.to_numeric(demograficos_df_uniq[field], errors='coerce')
field="SEMESTRE"
demograficos_df_uniq[field] = pd.to_numeric(demograficos_df_uniq[field], errors='coerce')
field="PROMEDIOSEMESTRE"
demograficos_df_uniq[field] = demograficos_df_uniq[field].astype(str).str.replace(',', '.', regex=False)
demograficos_df_uniq[field] = pd.to_numeric(demograficos_df_uniq[field], errors='coerce')
field="EDAD2"
demograficos_df_uniq[field] = pd.to_numeric(demograficos_df_uniq[field], errors='coerce')

In [19]:
numeric_vars = ["ESTRATO", "SEMESTRE", "PROMEDIOSEMESTRE", "EDAD2"]
category_vars = ["PROGRAMA", "GENERO", "CIUDADRESIDENCIA","CIUDADNACIMIENTO"]
target_var="target"
features=numeric_vars + category_vars

Conversion de las variables categoricas a numericas utilizando codificacion, para asegurar que todas las variables de entrada al modelo sean numericas.

In [20]:
def get_df(df, target_var="target", numeric_vars=[], category_vars=[]):
  df = df.copy()
  label_encoder = LabelEncoder()
  for col in category_vars:
      df[col] = label_encoder.fit_transform(df[col].astype(str))
  df=df[ numeric_vars + category_vars+ [target_var]]
  return df

In [21]:
df=demograficos_df_uniq
df=get_df(df, target_var, numeric_vars, category_vars)
df.sample(3)

,ESTRATO,SEMESTRE,PROMEDIOSEMESTRE,EDAD2,PROGRAMA,GENERO,CIUDADRESIDENCIA,CIUDADNACIMIENTO,target
10877,2.0,8,4.7,22,1,0,102,179,1
54132,2.0,5,3.0,20,1,1,23,47,1
14924,3.0,8,4.3,23,4,1,23,47,1


* ***Identificacion valores nulos en las variables de entrada:***

In [22]:
df[[target_var]+features].isnull().sum() / df.shape[0]

target              0.000000
ESTRATO             0.000599
SEMESTRE            0.000000
PROMEDIOSEMESTRE    0.000000
EDAD2               0.000000
PROGRAMA            0.000000
GENERO              0.000000
CIUDADRESIDENCIA    0.000000
CIUDADNACIMIENTO    0.000000
dtype: float64

Se observa que solo 1 columna del dataset tiene valores nulos, sin embargo representan un porcentaje minimo, por lo anterior decidimos conservar todas las variables de entrada.

In [23]:
duplicated_rows = df.loc[df.duplicated(subset=features, keep=False)].shape[0]
duplicated_rows

1462

In [24]:
print(f"Duplicates: {(duplicated_rows/total_rows)*100:.4f}%")

Duplicates: 2.7577%


In [25]:
duplicated_rows = df.loc[df.duplicated(subset=features+["target"], keep=False)].shape[0]
duplicated_rows

1385

In [26]:
print(f"Duplicates: {(duplicated_rows/total_rows)*100:.4f}%")

Duplicates: 2.6124%




Existe un ~2.75% de registros con variables de entrada duplicadas, incluso con variable objetivo diferente. Al incluir la variable objetivo dentro del análisis de duplicados, se obtiene el ~2.61% registros duplicados adicionales. Para un porcentaje total de 5,36%, se decide eliminar los datos duplicados con el fin de no afectar en la funcion de estimacion.


In [27]:
df.drop_duplicates(subset=features, inplace=True)
df.drop_duplicates(subset=features+["target"], inplace=True)

In [28]:
df.shape

(9096, 9)

In [30]:
df.to_csv('./data/clean.csv', index=False)